In [1]:
from typing import List

In [2]:
class State:
    # Create a new state
    def __init__(self, route: List, distance: int = 0):
        self.route = route
        self.distance = distance
    # Compare states

    def __eq__(self, other):
        for i in range(len(self.route)):
            if(self.route[i] != other.route[i]):
                return False
        return True
    # Sort states

    def __lt__(self, other):
        return self.distance < other.distance
    # Print a state

    def __repr__(self):
        return ('({0},{1})\n'.format(self.route, self.distance))
    # Create a shallow copy

    def copy(self):
        return State(self.route, self.distance)
    # Create a deep copy

    def deepcopy(self):
        return State(copy.deepcopy(self.route), copy.deepcopy(self.distance))
    # Update distance

    def update_distance(self, matrix, home):

        # Reset distance
        self.distance = 0
        # Keep track of departing city
        from_index = home
        # Loop all cities in the current route
        for i in range(len(self.route)):
            self.distance += matrix[from_index][self.route[i]]
            from_index = self.route[i]
        # Add the distance back to home
        self.distance += matrix[from_index][home]


In [3]:
# This class represent a city (used when we need to delete cities)
class City:
    # Create a new city
    def __init__(self, index: int, distance: int):
        self.index = index
        self.distance = distance
    # Sort cities

    def __lt__(self, other):
        return self.distance < other.distance

In [4]:
# Get the best random solution from a population
def get_random_solution(matrix: List, home: int, city_indexes: List, size: int, use_weights=False):
    # Create a list with city indexes
    cities = city_indexes.copy()
    # Remove the home city
    cities.pop(home)
    # Create a population
    population = []
    for i in range(size):
        if(use_weights == True):
            state = get_random_solution_with_weights(matrix, home)
        else:
            # Shuffle cities at random
            random.shuffle(cities)
            # Create a state
            state = State(cities[:])
            state.update_distance(matrix, home)
        # Add an individual to the population
        population.append(state)
    # Sort population
    population.sort()
    # Return the best solution
    return population[0]


In [5]:
# Get best solution by distance
def get_best_solution_by_distance(matrix: List, home: int):

    # Variables
    route = []
    from_index = home
    length = len(matrix) - 1
    # Loop until route is complete
    while len(route) < length:
        # Get a matrix row
        row = matrix[from_index]
        # Create a list with cities
        cities = {}
        for i in range(len(row)):
            cities[i] = City(i, row[i])
        # Remove cities that already is assigned to the route
        del cities[home]
        for i in route:
            del cities[i]
        # Sort cities
        sorted = list(cities.values())
        sorted.sort()
        # Add the city with the shortest distance
        from_index = sorted[0].index
        route.append(from_index)
    # Create a new state and update the distance
    state = State(route)
    state.update_distance(matrix, home)
    # Return a state
    return state

In [6]:
# Get a random solution by using weights
def get_random_solution_with_weights(matrix: List, home: int):
    # Variables
    route = []
    from_index = home
    length = len(matrix) - 1
    # Loop until route is complete
    while len(route) < length:
        # Get a matrix row
        row = matrix[from_index]
        # Create a list with cities
        cities = {}
        for i in range(len(row)):
            cities[i] = City(i, row[i])
        # Remove cities that already is assigned to the route
        del cities[home]
        for i in route:
            del cities[i]
        # Get the total weight
        total_weight = 0
        for key, city in cities.items():
            total_weight += city.distance
        # Add weights
        weights = []
        for key, city in cities.items():
            weights.append(total_weight / city.distance)
        # Add a city at random
        from_index = random.choices(list(cities.keys()), weights=weights)[0]
        route.append(from_index)
    # Create a new state and update the distance
    state = State(route)
    state.update_distance(matrix, home)
    # Return a state
    return state

In [7]:
# Mutate a solution
def mutate(matrix: List, home: int, state: State, mutation_rate: float = 0.01):

    # Create a copy of the state
    mutated_state = state.deepcopy()
    # Loop all the states in a route
    for i in range(len(mutated_state.route)):
        # Check if we should do a mutation
        if(random.random() < mutation_rate):
            # Swap two cities
            j = int(random.random() * len(state.route))
            city_1 = mutated_state.route[i]
            city_2 = mutated_state.route[j]
            mutated_state.route[i] = city_2
            mutated_state.route[j] = city_1
    # Update the distance
    mutated_state.update_distance(matrix, home)
    # Return a mutated state
    return mutated_state

In [8]:
# Hill climbing algorithm
def hill_climbing(matrix: List, home: int, initial_state: State, max_iterations: int, mutation_rate: float = 0.01):
    # Keep track of the best state
    best_state = initial_state
    # An iterator can be used to give the algorithm more time to find a solution
    iterator = 0
    # Create an infinite loop
    while True:
        # Mutate the best state
        neighbor = mutate(matrix, home, best_state, mutation_rate)
        # Check if the distance is less than in the best state
        if(neighbor.distance >= best_state.distance):
            iterator += 1
            if (iterator > max_iterations):
                break
        if(neighbor.distance < best_state.distance):
            best_state = neighbor
    # Return the best state
    return best_state

In [9]:
# The main entry point for this module
def main():
    # Cities to travel
    cities = ['New Delhi', 'Gurgaon', 'Chennai',
              'Kolkata', 'Mumbai', 'Bangalore']
    city_indexes = [0, 1, 2, 3, 4, 5]
    # Index of start location
    home = 2  # Chicago
    # Max iterations
    max_iterations = 1000
    # Distances in miles between cities, same indexes (i, j) as in the cities array
    matrix = [[0, 31, 2176, 1471, 1418, 2141],
              [31, 0, 2177, 1496, 1390, 2142],
              [2176, 2177, 0, 1682, 1335, 338],
              [1471, 1496, 1682, 0, 1996, 1892],
              [1418, 1390, 1335, 1996, 0, 993],
              [2141, 2142, 338, 1892, 993, 0]]
    # Get the best route by distance
    state = get_best_solution_by_distance(matrix, home)
    print('-- Best solution by distance --')
    print(cities[home], end='')
    for i in range(0, len(state.route)):
        print(' -> ' + cities[state.route[i]], end='')
    print(' -> ' + cities[home], end='')
    print('\n\nTotal distance: {0} km'.format(state.distance))
    print()
    # Run hill climbing to find a better solution
    state = get_best_solution_by_distance(matrix, home)
    state = hill_climbing(matrix, home, state, 1000, 0.1)
    print('-- Hill climbing solution --')
    print(cities[home], end='')
    for i in range(0, len(state.route)):
        print(' -> ' + cities[state.route[i]], end='')
    print(' -> ' + cities[home], end='')
    print('\n\nTotal distance: {0} km'.format(state.distance))
    print()
    abcx = input("A")

In [ ]:
import random
import copy
main()

-- Best solution by distance --
Chennai -> Bangalore -> Mumbai -> Gurgaon -> New Delhi -> Kolkata -> Chennai

Total distance: 5905 km

-- Hill climbing solution --
Chennai -> Bangalore -> Mumbai -> Gurgaon -> New Delhi -> Kolkata -> Chennai

Total distance: 5905 km

